In [14]:
import gym
import minihack
import matplotlib.pyplot as plt
import numpy as np
from nle import nethack
from calm.environment import ShaperWrapper

actions = [
    nethack.CompassCardinalDirection.N,
    nethack.CompassCardinalDirection.E,
    nethack.CompassCardinalDirection.S,
    nethack.CompassCardinalDirection.W,
    nethack.Command.PICKUP,
    nethack.Command.APPLY,
]
env = gym.make(
    "MiniHack-River-Narrow-v0",
    # "MiniHack-KeyRoom-S5-v0",
    actions=actions,
    observation_keys=(
        "pixel",
        "pixel_crop",
        "chars",
        "chars_crop",
        "message",
        "blstats",
        "screen_descriptions",
        "inv_glyphs",
        "inv_letters",
        "inv_oclasses",
        "inv_strs"
    ),
    max_episode_steps=100,
    # seeds=[[0]] * 3,
    # num_envs=3,
    # asynchronous=True,
    # wrappers=ShaperWrapper
)

In [16]:
obs, info = env.reset()
env.render()


You are lucky!  Full moon tonight.                                              
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                          .....@............}                                   
                          .  `.........`    }                                   
                             .............                                      
                           ................`                                    
                          .

/home/uceeepi/miniforge3/envs/lfrl/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


In [ ]:
for action in [2, 3, 4]:
    obs, reward, term, trunc, info = env.step([action] * 3)
    env.render()
    print("Reward:", reward)

In [ ]:
for action in [1, 1, 1, 0, 0, 0, 5]:
    obs, reward, term, trunc, info = env.step(action)
    env.render()
    print("Reward:", reward)

In [60]:
obs["inv_glyphs"]

array([1935, 1923, 2019, 2191, 2103, 2098, 5976, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976],
      dtype=int16)

In [91]:
obs["inv_glyphs"]

array([1935, 1923, 2019, 2180, 2103, 2098, 2102, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976],
      dtype=int16)

In [65]:
a = np.array([1935, 1923, 2019, 2191, 2103, 2098, 5976, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976],
      dtype=int)

b = np.array([1935, 1923, 2019, 2191, 2103, 2098, 2102, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976,
       5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976, 5976],
      dtype=int)

In [67]:
b[6]

2102

In [93]:
2102 in obs["inv_glyphs"]

False

In [169]:
import jax
import jax.numpy as jnp

key = jax.random.PRNGKey(0)

@jax.jit
def f(x):
    return jax.random.randint(key, (5,), minval=0, maxval=x)

y = jnp.asarray(2)
x = jax.lax.cond(y == jnp.asarray(1), lambda: jnp.asarray(2), lambda: jnp.asarray(10))

f(x)

Array([8, 1, 3, 8, 8], dtype=int32)

In [170]:
jnp.arange(1)

Array([0], dtype=int32)

In [162]:
import jax
import jax.numpy as jnp

def redistribute_rewards(pytree):
    rewards = pytree["reward"]
    times = pytree["time"]
    
    reward_mask = jnp.cumsum(rewards[::-1])[::-1]
    time_mask = jnp.cumsum(times == 0)
    # print(reward_mask, time_mask)
    
    for i in jnp.unique(time_mask):
        episode_mask = time_mask == i
        print(episode_mask)
        r = rewards * (episode_mask)
        rewards = jnp.where(episode_mask, r.sum() / sum(episode_mask), rewards)
    return rewards
    
# Example usage
pytree = {"reward": jnp.asarray([0, 1, 0, 0, 0, 0, 0, 0, 0]), 
          "time": jnp.asarray([4, 5, 0, 1, 2, 3, 0, 1, 2]),
          "term": jnp.asarray([0, 1, 0, 0, 0, 1, 0, 0, 0])}

# pytree = {"reward": jnp.asarray([1, 0, 0, 0, 0, 0, 0, 0, 0]), 
#           "time": jnp.asarray([4, 5, 0, 1, 2, 3, 0, 1, 2])}

# pytree = {"reward": jnp.asarray([0, 0, 0, 0, 0, 0, 0, 0, 1]), 
#           "time": jnp.asarray([4, 5, 0, 1, 2, 3, 0, 1, 2])}

# pytree = {"reward": jnp.asarray([0, 0, 0, 0, 0, 0, 0, 0, 1]), 
#           "time": jnp.asarray([4, 5, 0, 1, 2, 3, 0, 1, 2])}

# pytree = {"reward": jnp.asarray([0, 0, 0, 0, 0, 0, 0, 0, 1]), 
#           "time": jnp.asarray([4, 5, 0, 1, 2, 3, 0, 1, 2])}

print(redistribute_rewards(pytree))

[ True  True False False False False False False False]
[False False  True  True  True  True False False False]
[False False False False False False  True  True  True]
[0.5 0.5 0.  0.  0.  0.  0.  0.  0. ]


In [36]:
import os
import pickle
from calf.io import load_pickle_minibatches, load_pickle_stream

In [2]:
path = "/scratch/uceeepi/calf/experiment_2/ann_full.pkl"
stream = load_pickle_stream(path)

In [17]:
for annotation in stream[:100]:
    for key in annotation.parsed:
        obs, action = key
        if int(action) == 4 or int(action) == 5:
            print(str(key).encode().decode("unicode_escape"), annotation.parsed[key])

('
                                                                                                                                                             
                                                                                                                                                             
                                                                                                                                                             
                                                                                                                                                             
                                                                                                                                                             
                                                                                                                                                             
                                                 